# Préparation

## Installation des packages

In [1]:
import pymongo
from pymongo import MongoClient
import json
import ujson
from bson import json_util
from bson.json_util import loads

## Connexion à la base de données

In [2]:
client = MongoClient(host="localhost", port=27017)
db = client["20_Warehouses"]

# Installation des fichiers de normalisation

## Installation de la collection `Customer_20`

In [3]:
Customer_20 = db["Customer_20"]

data = []
with open("../1_Warehouse/Normalization/Customer_1.json") as f:
    for line in f:
        data.append(json.loads(line))
with open("../10_Warehouses/Normalization/Customer_2_10.json") as f:
    for line in f:
        data.append(json.loads(line))
with open("./Normalization/Customer_11_16.json") as f:
    for line in f:
        data.append(json.loads(line))
with open("./Normalization/Customer_17_20.json") as f:
    for line in f:
        data.append(json.loads(line))
Customer_20.insert_many(data)
print("Fichiers Customer_1.json, Customer_2_10.json, \
Customer_11_16.json et Customer_17_20.json sont insérés avec succès.")

Fichiers Customer_1.json, Customer_2_10.json, Customer_11_16.json et Customer_17_20.json sont insérés avec succès.


## Installation de la collection `Order_20`

In [4]:
Order_20 = db["Order_20"]

data = []
with open("../10_Warehouses/Normalization/Order_1_10.json") as f:
    for line in f:
        data.append(json.loads(line))
with open("./Normalization/Order_11_20.json") as f:
    for line in f:
        data.append(json.loads(line))
Order_20.insert_many(data)
print("Fichiers Order_1_10.json et Order_11_20.json sont insérés avec \
succès.")

Fichiers Order_1_10.json et Order_11_20.json sont insérés avec succès.


## Installation de la collection `Order_Line_20`

In [5]:
Order_Line_20 = db["Order_Line_20"]

data = []
with open("./Normalization/Order_Line_1_20.json") as f:
    for line in f:
        data.append(json.loads(line))
Order_Line_20.insert_many(data)
print("Fichier Order_Line_1_20.json est inséré avec succès.")

Fichier Order_Line_1_20.json est inséré avec succès.


# Adjonction des index des collections installées

## Adjonction des index de la collection `Customer_20`

In [6]:
result = db.Customer_20.create_index([('c_id', pymongo.ASCENDING)])
result = db.Customer_20.create_index([('c_d_id', pymongo.ASCENDING)])
result = db.Customer_20.create_index([('c_w_id', pymongo.ASCENDING)])
list(db.Customer_20.index_information())

['_id_', 'c_id_1', 'c_d_id_1', 'c_w_id_1']

## Adjonction des index de la collection `Order_20`

In [7]:
result = db.Order_20.create_index([('o_id', pymongo.ASCENDING)])
result = db.Order_20.create_index([('o_d_id', pymongo.ASCENDING)])
result = db.Order_20.create_index([('o_w_id', pymongo.ASCENDING)])
result = db.Order_20.create_index([('o_c_id', pymongo.ASCENDING)])
list(db.Order_20.index_information())

['_id_', 'o_id_1', 'o_d_id_1', 'o_w_id_1', 'o_c_id_1']

## Adjonction des index de la collection `Order_Line_20`

In [8]:
result = db.Order_Line_20.create_index([('ol_id', pymongo.ASCENDING)])
result = db.Order_Line_20.create_index([('ol_d_id', pymongo.ASCENDING)])
result = db.Order_Line_20.create_index([('ol_w_id', pymongo.ASCENDING)])
list(db.Order_Line_20.index_information())

['_id_', 'ol_id_1', 'ol_d_id_1', 'ol_w_id_1']

# Semi-dénormalisation

## Création du fichier au format JSON

In [9]:
i = 1
while i <= 3000:
    i_str = str(i)
    ujson.dumps(
        db.Order_20.aggregate(
            [{
                "$match": {
                    "o_id": i_str
                }
            },
             {
                 "$lookup": {
                     "from":
                     "Order_Line_20",
                     "let": {
                         "o_id": "$o_id",
                         "o_d_id": "$o_d_id",
                         "o_w_id": "$o_w_id"
                     },
                     "pipeline": [{
                         "$match": {
                             "$expr": {
                                 "$and": [{
                                     "$eq": ["$ol_id", "$$o_id"]
                                 }, {
                                     "$eq": ["$ol_d_id", "$$o_d_id"]
                                 }, {
                                     "$eq": ["$ol_w_id", "$$o_w_id"]
                                 }]
                             }
                         }
                     }],
                     "as":
                     "order_lines"
                 }
             }, {
                 "$out": "tmp"
             }]))
    cursor = db.tmp.find({})
    file = open("./Denormalization/Semi_20.json", "a")
    for document in cursor:
        file.write(json.dumps(document, default=json_util.default))
        file.write('\n')
    i += 1

print("Fichier Semi_20.json est créé avec succès.")

Fichier Semi_20.json est créé avec succès.


##  Installation de la collection `Semi_20`

In [10]:
Semi_20 = db["Semi_20"]

data = []
with open("./Denormalization/Semi_20.json", "r") as f:
    for line in f:
        data.append(loads(line))
Semi_20.insert_many(data)
print("Fichier Semi_20.json est inséré avec succès.")

Fichier Semi_20.json est inséré avec succès.


## Monstration d’un échantillon

In [11]:
list(
    db.Semi_20.find({
        "$and": [{
            "o_id": "1"
        }, {
            "o_c_id": "1"
        }, {
            "o_d_id": "1"
        }, {
            "o_w_id": "20"
        }]
    }, {
        "_id": 0,
    }))

[{'o_id': '1',
  'o_d_id': '1',
  'o_w_id': '20',
  'o_c_id': '1',
  'o_entry_d': '2018-07-20 00:00:00',
  'o_carrier_id': '7',
  'o_ol_cnt': '6',
  'o_all_local': '1',
  'order_lines': [{'_id': ObjectId('5c4dab1ce3ecda0fe0f02f4f'),
    'ol_id': '1',
    'ol_d_id': '1',
    'ol_w_id': '20',
    'ol_number': '1',
    'ol_i_id': '79265',
    'ol_supply_w_id': '20',
    'ol_delivery_d': '2018-07-20 00:00:00',
    'ol_quantity': 5,
    'ol_amount': 99.26,
    'ol_dist_info': 'sCUbLg8BXXqcCSXRS98wTZGp'},
   {'_id': ObjectId('5c4dab1ce3ecda0fe0f02f50'),
    'ol_id': '1',
    'ol_d_id': '1',
    'ol_w_id': '20',
    'ol_number': '2',
    'ol_i_id': '51701',
    'ol_supply_w_id': '20',
    'ol_delivery_d': '2018-07-20 00:00:00',
    'ol_quantity': 5,
    'ol_amount': 23.72,
    'ol_dist_info': '5AnUqMvMEEN6SPI9YQFPYfqE'},
   {'_id': ObjectId('5c4dab1ce3ecda0fe0f02f51'),
    'ol_id': '1',
    'ol_d_id': '1',
    'ol_w_id': '20',
    'ol_number': '3',
    'ol_i_id': '95150',
    'ol_supply_w_id'

## Adjonction des index de la collection `Semi_20`

In [12]:
result = db.Semi_20.create_index([('o_id', pymongo.ASCENDING)])
result = db.Semi_20.create_index([('o_d_id', pymongo.ASCENDING)])
result = db.Semi_20.create_index([('o_w_id', pymongo.ASCENDING)])
result = db.Semi_20.create_index([('o_c_id', pymongo.ASCENDING)])
list(db.Semi_20.index_information())

['_id_', 'o_id_1', 'o_d_id_1', 'o_w_id_1', 'o_c_id_1']

# Dénormalisation

## Création du fichier au format JSON

In [13]:
i = 1
while i <= 3000:
    i_str = str(i)
    ujson.dumps(
        db.Customer_20.aggregate(
            [{
                "$match": {
                    "c_id": i_str
                }
            },
             {
                 "$lookup": {
                     "from":
                     "Semi_20",
                     "let": {
                         "c_id": "$c_id",
                         "c_d_id": "$c_d_id",
                         "c_w_id": "$c_w_id"
                     },
                     "pipeline": [{
                         "$match": {
                             "$expr": {
                                 "$and": [{
                                     "$eq": ["$o_c_id", "$$c_id"]
                                 }, {
                                     "$eq": ["$o_d_id", "$$c_d_id"]
                                 }, {
                                     "$eq": ["$o_w_id", "$$c_w_id"]
                                 }]
                             }
                         }
                     }],
                     "as":
                     "customer_order"
                 }
             }, {
                 "$out": "tmp"
             }]))
    cursor = db.tmp.find({})
    file = open("./Denormalization/Denormal_20.json", "a")
    for document in cursor:
        file.write(json.dumps(document, default=json_util.default))
        file.write('\n')
    i += 1
print("Fichier Denormal_20.json est créé avec succès.")

Fichier Denormal_20.json est créé avec succès.


##  Installation de la collection `Denormal_20`

In [14]:
Denormal_20 = db["Denormal_20"]

data = []
with open("./Denormalization/Denormal_20.json", "r") as f:
    for line in f:
        data.append(loads(line))
Denormal_20.insert_many(data)
print("Fichier Denormal_20.json est inséré avec succès.")

Fichier Denormal_20.json est inséré avec succès.


## Monstration d’un échantillon

In [15]:
list(
    db.Denormal_20.find({
        "$and": [{
            "c_id": "1"
        }, {
            "c_d_id": "1"
        }, {
            "c_w_id": "20"
        }]
    }, {
        "_id": 0,
    }))

[{'c_id': '1',
  'c_d_id': '1',
  'c_w_id': '20',
  'c_first': 'MTqNFwUjPwkJ',
  'c_middle': 'OE',
  'c_last': 'BARBARBAR',
  'c_street_1': 'JC1fbfxGl0Vnb0B',
  'c_street_2': 'jK7DmeQEHJFo',
  'c_city': 'nLIOnDBnGx',
  'c_state': 'vW',
  'c_zip': 'sahR9VXsQ',
  'c_phone': '0898660987571287',
  'c_since': '2018-07-20 00:00:00',
  'c_credit': 'GC',
  'c_credit_lim': '50000',
  'c_discount': '0.05',
  'c_balance': -10.0,
  'c_ytd_payment': 10.0,
  'c_payment_cnt': 1,
  'c_delivery_cnt': 0,
  'c_data': 'JDcZmffHaCz34Oyo9CxUtPlufDk1tQbjbKDftA6u2rV71UT5D4tnP0OvTn4P3MrOVDDArFfhjTCJbN3RNNYLJNjy2mPuXbfZKmEi8zE8Bu1FUJK3K5PPv5LHBPv2b0ye6vtj8timEo587dBF28H5u3PblJhEfJuQ65kG5N7Og7HFIAvIUfpTaXUXU2ItAiwyUm8pghLUSo4JcOgXOgibCtfN7ClqIIsWyE6wpVRSYyCeXrCPsFuOSQyaLPH2NtjU4HYtrCIJJbaOonqI6to5hoKd8M1LQBU1mQgzjiYgzp7orayyFoOJRHO41TctJzmGMk6DFLl6bcYVg0He9ZjoMezMe2Ed4SIk9PuJnmm6JV0VYnNlTu3iWHcOTgiuTo4Bt1fVI5VGVUqxYGsq4HlCOCcQhANmh73fcEzfv5d2Y9j4O40tM',
  'customer_order': [{'_id': ObjectId('5c4da9d7e3ecda0fe098

## Adjonction des index de la collection `Denormal_20`

In [16]:
result = db.Denormal_20.create_index([('c_id', pymongo.ASCENDING)])
result = db.Denormal_20.create_index([('c_d_id', pymongo.ASCENDING)])
result = db.Denormal_20.create_index([('c_w_id', pymongo.ASCENDING)])
list(db.Denormal_20.index_information())

['_id_', 'c_id_1', 'c_d_id_1', 'c_w_id_1']